# Jupyter Notebooks

* Interactive computational environment for working with **Ju**lia, **P**ython, and **R** and more (40+ languages)
* Composed of input and output cells that can contain code or text (via Markdown) and produce a range of outputs
* Enables [literate programming](https://en.wikipedia.org/wiki/Literate_programming) paradigm of text and documentation interleaved with code

**_Pro tip: Use more modern JupyterLab over classic Jupyter Notebook_**


**_Bonus pro tip: Get comfortable with Jupyter's keybindings_**

There are also a range of useful macros that are extensions beyond Python features.

In [ ]:
# these macros make Jupyter re-run imported libraries when it detects their contents on disk have changed
%load_ext autoreload
%autoreload 2

## Data: Weekly Payroll Jobs and Wages in Australia

Data provided by the Australian Bureau of Statistics pertaining to weekly numbers of jobs with payroll data. This is based on the Australian Tax Office's single touch payroll data, which is how most businesses report salaries and wages, pay as you go (PAYG) withholding, and superannuation.

[Weekly Jobs and Wages Index for Week ending 21/03/2021.](https://www.abs.gov.au/statistics/labour/earnings-and-work-hours/weekly-payroll-jobs-and-wages-australia/week-ending-27-march-2021#data-download)


[Excel file used -- Table 5: Sub-state - Payroll jobs indexes](https://www.abs.gov.au/statistics/labour/earnings-and-work-hours/weekly-payroll-jobs-and-wages-australia/week-ending-27-march-2021/6160055001_DO005.xlsx)

## Pandas Data Wrangling

Pandas is a powerful Swiss Army knife for doing data manipulation and analysis in Python.

The two primary data structures it makes available are the `DataFrame`, for working with two-dimensional tabular data, and the `Series` for working with one-dimensional arrays.

We'll need to install the `openpyxl` in order to read Excel files.

In [ ]:
# you can run shell commands by prefixing with `!`
! pip install openpyxl

In [ ]:
import pandas as pd

df = pd.read_excel("data/6160055001_DO005.xlsx", sheet_name=1)
df.shape

Note that Jupyter defaults to displaying the output of the previous cell on execution.

In [ ]:
df.head()

Let's use the `dtale` package to better explore the DataFrame.

In [ ]:
import dtale

dtale.show(df)

In [ ]:
jobs_raw_df = pd.read_excel(
    "data/6160055001_DO005.xlsx",
    sheet_name=1,  # zero-indexed, so this is the second sheet!
    usecols="A:BO",
    skiprows=5,
    skipfooter=2,
)

jobs_raw_df.head()

This data is in [wide](https://en.wikipedia.org/wiki/Wide_and_narrow_data) format. We need to make it into long (or narrow) form, where each row contains a single record.

We also need to split the first two columns into their codes and names.

In [ ]:
# get a DataFrame of all columns except the first two
date_cols_df = jobs_raw_df.iloc[:, 2:]

# split col 1 into codes and names
ste_cols_df = (
    jobs_raw_df[jobs_raw_df.columns[0]]
    .str.split(r"\. ", expand=True)
    .rename(columns={0: "STE_CODE16", 1: "STE_NAME16"})
)

# split col 2 into codes and names
sa4_cols_df = (
    jobs_raw_df[jobs_raw_df.columns[1]]
    .str.split(r"\. ", expand=True)
    .rename(columns={0: "SA4_CODE16", 1: "SA4_NAME16"})
)

# combine the 3 sets of columns into a single DataFrame, and then use the melt method
# to convert it into long format
jobs_df = pd.concat([ste_cols_df, sa4_cols_df, date_cols_df], axis=1).melt(
    id_vars=["STE_CODE16", "STE_NAME16", "SA4_CODE16", "SA4_NAME16"],
    var_name="Date",
    value_name="Index",
)

jobs_df

In [ ]:
jobs_df.dtypes

Let's get the mean index for each state across all dates:

In [ ]:
states_jobs = jobs_df.groupby(["STE_NAME16", "Date"])["Index"].mean()
states_jobs

Now get the mean of the state means for each date:

In [ ]:
country_jobs = states_jobs.mean(level="Date")
country_jobs

# Visualising with Pandas & Matplotlib

In [ ]:
# convert Series from previous cell to DataFrame for easier plotting

country_jobs_df = country_jobs.reset_index()
country_jobs_df

### Use Pandas' API to make Matplotlib plot

We're going to start by using Pandas to plot the wage Index over time.

Pandas uses Matplotlib under the hood for plotting. Pandas' [plotting backend](https://pandas.pydata.org/pandas-docs/dev/user_guide/visualization.html#plotting-backends) feature allows other libraries to be used.

Current plotting backends available:

* [Plotly](https://plotly.com/python/pandas-backend/)
* [Bokeh](https://github.com/PatrikHlobil/Pandas-Bokeh)

In [ ]:
# tell Jupyter to render Matplotlib
%matplotlib inline

country_jobs_df.plot(x="Date", y="Index", title="Weekly Payroll Jobs and Wages Index");

**_Pro tip: Always title your plots!_**

## Improving Aesthetics of Matplotlib plots

**Approach 1: use a theme**

In [ ]:
import matplotlib.pyplot as plt

plt.style.available

In [ ]:
plt.style.use("ggplot")

country_jobs_df.plot(x="Date", y="Index", title="Weekly Payroll Jobs and Wages Index");

**Approach 2: Increase the DPI**

In [ ]:
fig = plt.figure(dpi=300, figsize=(15, 5))

country_jobs_df.plot(
    x="Date", y="Index", ax=plt.gca(), title="Weekly Payroll Jobs and Wages Index"
);

### Use ipympl Widget for interactivity

_Note: this requires the ipympl package to be installed_

Let's plot the Index of all states. This is simple with Pandas, but requires our data to be in wide format.

In [ ]:
%matplotlib widget

# use the unstack method to get it into wide format

states_jobs.unstack(level=0).plot(
    figsize=(15, 5), title="Weekly Payroll Jobs and Wages Index by State"
);

What if our data was in long (tidy) format?

This is harder with Pandas. Easier to use a different tool.

In [ ]:
states_jobs_df = states_jobs.reset_index()
states_jobs_df

## Other plotting Options

### Seaborn

Like Pandas, Seaborn is built on top of Matplotlib. It provides an API that is particularly well-suited for making more scientific/statistically oriented plots.

See the [Seaborn gallery](https://seaborn.pydata.org/examples/index.html) for examples.

In [ ]:
import seaborn as sns

plt.figure(figsize=(13, 6))

sns.lineplot(
    data=states_jobs_df,
    x="Date",
    y="Index",
    hue="STE_NAME16",
).set(title="Weekly Payroll Jobs and Wages Index by State");

### Plotly

Plotly is powered by a JavaScript-based visualisation library that gives you interactive plots out of the box.

There is a lower-level Python interface for defining plots as [`Figure` objects](https://plotly.com/python/figure-structure), as well as the more concise [Plotly Express](https://plotly.com/python/plotly-express/) that produces `Figure` objects with much less coding required.

A gallery of examples can be found [here](https://plotly.com/python/).

In [ ]:
import plotly.express as px

px.line(
    states_jobs_df,
    x="Date",
    y="Index",
    color="STE_NAME16",
    title="Weekly Payroll Jobs and Wages Index by State",
    width=1200,
    height=500,
)

### Adding Country-level mean

In [ ]:
states_and_country_df = pd.concat(
    [states_jobs_df, country_jobs_df.assign(STE_NAME16="AUS")]
)
state_names = list(states_jobs_df["STE_NAME16"].unique())

px.line(
    states_and_country_df,
    x="Date",
    y="Index",
    color="STE_NAME16",
    title="Weekly Payroll Jobs and Wages Index by State",
    color_discrete_map={"AUS": "black"},
    category_orders={"STE_NAME16": ["AUS"] + state_names},
    line_dash="STE_NAME16",
    line_dash_sequence=["dot"] + ["solid" for _state in state_names],
    width=1200,
    height=500,
)

### Other options to consider
* Altair (similar to ggplot, with good interactive API)
* Holoviews (has Matplotlib, Bokeh, and Plotly backends)


# TODO: graphic of libraries

# Spatial Visualisation

A useful tool for working with spatial data is [Geopandas](https://geopandas.org/).

It provides `GeoSeries` and `GeoDataFrame` data structures, like Pandas' `Series` and `DataFrame`, but for working with shaply objects: 
* Points / Multi-Points
* Lines / Multi-Lines
* Polygons / Multi-Polygons

In [ ]:
import geopandas as gpd

gdf = gpd.read_file("data/sa4_2016_aust_shape/SA4_2016_AUST.shp")

gdf

Like Pandas, Geopandas can produce plots using Matplotlib.

## PLotting a Choropleth with Geopandas 

Let's plot a spatial heat map plat (choropleth) of mean index by SA4 region.

We'll wrap it up in a function that takes a data as an input parameter, and filters the data down to results for that date and plots the choropleth.

In [ ]:
import contextily as cx


def plot_wage_chloropleth(sa4_gdf, jobs_df, date):
    """Plot a chloropleth map of jobs Index for a given date"""
    # remove records with empty geometry, and filter data to current month,
    # then join with geo data
    sa4_gdf = sa4_gdf[~sa4_gdf["geometry"].isnull()]
    filtered_df = jobs_df[jobs_df["Date"] == date]
    sa4_gdf = sa4_gdf.merge(filtered_df, on="SA4_CODE16", validate="one_to_one")

    fig, ax = plt.subplots()
    sa4_gdf.plot(
        ax=ax,
        edgecolor="black",
        column="Index",
        vmin=jobs_df["Index"].min(),
        vmax=jobs_df["Index"].max(),
    ).set(title="Australian Jobs and Wages Index")

    # set the basemap tiles with contexily
    cx.add_basemap(ax, crs=gdf.crs.to_string(), source=cx.providers.CartoDB.Voyager)
    ax.axis("off")


plot_wage_chloropleth(gdf, jobs_df, "2020-01-04")

**_Pro tip: Move code for producing distinct plots into function_**

## Plotting a Choropleth with Folium

Folium is a Python wrapper around the Leaflet JavaScript library for producing web-based interactive maps.

In [ ]:
import folium


def plot_wage_chloropleth_folium(sa4_gdf, jobs_df, date):
    """Plot a chloropleth map of jobs Index for a given date"""
    # remove records with empty geometry, and filter data to current month
    sa4_gdf = sa4_gdf[~sa4_gdf["geometry"].isnull()]
    filtered_df = jobs_df[jobs_df["Date"] == date]

    folium_map = folium.Map(location=[-22, 133], zoom_start=5)

    choropleth = folium.Choropleth(
        geo_data=sa4_gdf,
        data=filtered_df,
        columns=["SA4_NAME16", "Index"],
        key_on="feature.properties.SA4_NAME16",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Wage Index",
        highlight=True,
    )
    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(["SA4_NAME16"], labels=False)
    )
    choropleth.add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    return folium_map


plot_wage_chloropleth_folium(gdf, jobs_df, "2020-01-04")

Note that in the above solution, the color map is not being mapped to the ranges of the data from the whole year. This seems to be trickier to do with Folium's choropleth than GeoPandas' choropleth.

Working hypothesis is that for choropleth's needing any kind of customisation, the best way to approach is to user the lower-level `folium.GeoJson` class and set the `style_function` and `highlight_function` parameters.

## Other options to consider:

* [ipyleaflet](https://ipyleaflet.readthedocs.io/), for creating Leaflet plots along with interactive ipywidget controls.
* [Geoviews](https://geoviews.org/) (related to the Holoviews project)
* [Plotly](https://plotly.com/python/mapbox-layers/) (some integration with Mapbox requires access token)
* [pydeck](https://deckgl.readthedocs.io/en/latest) (based on [deck.gl](https://deck.gl) and with Jupyter widget support)
* [Datashader](https://datashader.org/) for rendering spatial visualisations of really large datasets